In [129]:
import matplotlib.pyplot as plt
import os
import librosa
import numpy as np
from pydub import AudioSegment
import h5py

In [165]:
def plot_mfcc_spectrogram(log_mel_spectrogram, sr):
    plt.figure(figsize=(10, 5))
    librosa.display.specshow(log_mel_spectrogram, sr=sr, hop_length=256, x_axis='time', y_axis='mel')
    print(log_mel_spectrogram.shape)
    plt.colorbar(format='%+2.0f dB')
    plt.title('Log Mel Spectrogram')
    plt.show()

def audio_to_spectrogram(dest_dir):
    h5_arr = []
    with h5py.File(dest_dir, 'w') as hf:
        # Iterate over the FLAC files in the source directory
        for root, _, files in os.walk("../iu", topdown=False):
            for file_name in files:
                if file_name.endswith('.flac') or file_name.endswith('.mp3'):
                    # Load the FLAC file
                    input_file = os.path.join(root, file_name)
                    audio, sr = librosa.load(input_file, sr=22050)
                    sec = sr * 3

                    # Generate the spectrogram batches
                    for i in range(audio.shape[0]//(sr * 3)):
                        batch = audio[i*sec:(i+1)*sec]
                        mel_spectrogram = librosa.feature.melspectrogram(y=batch, sr=sr, n_fft=4096, hop_length=512, n_mels=80)
                        log_mel_spectrogram = librosa.power_to_db(mel_spectrogram, ref=np.max)
                        h5_arr.append(log_mel_spectrogram)
                        ## hyper parameter:4096 / 512
                        ## larger nfft better for frequency resolution
                        ## higher hop length for time resolution
        
        h5_arr = np.array(h5_arr)
        hf.create_dataset("audio", data=log_mel_spectrogram)

In [166]:
audio_to_spectrogram('iu_spec.h5')

[src/libmpg123/id3.c:process_comment():584] error: No comment text / valid description?
[src/libmpg123/id3.c:process_comment():584] error: No comment text / valid description?
[src/libmpg123/id3.c:process_comment():584] error: No comment text / valid description?
[src/libmpg123/id3.c:process_comment():584] error: No comment text / valid description?
[src/libmpg123/id3.c:process_comment():584] error: No comment text / valid description?
[src/libmpg123/id3.c:process_comment():584] error: No comment text / valid description?
[src/libmpg123/id3.c:process_comment():584] error: No comment text / valid description?
[src/libmpg123/id3.c:process_comment():584] error: No comment text / valid description?
[src/libmpg123/id3.c:process_comment():584] error: No comment text / valid description?
[src/libmpg123/id3.c:process_comment():584] error: No comment text / valid description?
[src/libmpg123/id3.c:process_comment():584] error: No comment text / valid description?
[src/libmpg123/id3.c:process_ext

In [168]:
h5 = h5py.File("iu_spec.h5", 'r')
print(h5.get("audio"))

<HDF5 dataset "audio": shape (80, 130), type "<f4">
